### Web scrapping the https://www.opensecrets.org/races/election starting from TN district 7

In [1]:
#Import the packages and library
import requests
from bs4 import BeautifulSoup as BS
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import re

%matplotlib inline

import seaborn as sns
import datetime

## Reading TN District 07

In [2]:
URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'  #URL of the webpage

response = requests.get(URL)  #creating a response object pulling the URL
type(response)       # it is a requests.status.models.Response
response.status_code  #200 not broke read properly
response.text         # helps open the object to see it looks
soup = BS(response.text) #converts the response object in Beautiful object
#print(soup.prettify()) #to visualize how the conversion looks like and be able to read the class, tab etc where are the containers
soup.find('title')

<title>Tennessee District 07 2020 Race • OpenSecrets</title>

### we are have the soup so we can start finding the info and found out that Class 'Member--list-item' has all the members from the district contained in it.

In [3]:
#Finding all the members in the list item and 
list_item1 = soup.find('div', attrs={'class' : 'Members--list-item'}).text
list_item1

'\n\n\nMark Green (R) • Incumbent\n • Winner \n(69.9% of vote)\n\n\n\n\nRaised:\n$1,194,960\n\n\n\n\n\n\n\nSpent:\n$935,487\n\n\n\n\n\n\n\nCash on Hand:\n$287,889\n\n\n\n\n\n\n\n\nSource of Funds\n\n\n\nType of Contribution\nAmount\nPercentage\n\n\n\n\nSmall Individual Contributions (≤ $200)\n$0\n0.00%\n\n\nLarge Individual Contributions\n$819,151\n68.55%\n\n\nPAC Contributions*\n$171,900\n14.39%\n\n\nCandidate self-financing\n$0\n0.00%\n\n\nOther\n$203,909\n17.06%\n\n\n\nQuality of Disclosure\n\n\n\nType of Disclosure\nAmount\nPercentage\n\n\n\n\nFull Disclosure\n$1,061,024\n98.64%\n\n\nIncomplete Disclosure\n$2,676\n0.25%\n\n\nNo Disclosure\n$11,928\n1.11%\n\n\n\n\n'

#### List_item1 has many \n, , and dot that needes to be replaced so that we can run reggular expression on it so using the .replace function and assingn back to list_item1

In [4]:
list_item1 = list_item1.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
list_item1

'   Mark Green (R)  Incumbent   Winner  (69.9% of vote)     Raised: $1194960        Spent: $935487        Cash on Hand: $287889         Source of Funds    Type of Contribution Amount Percentage     Small Individual Contributions (≤ $200) $0 0.00%   Large Individual Contributions $819151 68.55%   PAC Contributions* $171900 14.39%   Candidate self-financing $0 0.00%   Other $203909 17.06%    Quality of Disclosure    Type of Disclosure Amount Percentage     Full Disclosure $1061024 98.64%   Incomplete Disclosure $2676 0.25%   No Disclosure $11928 1.11%     '

#### Using regular expression and pulling out the candidate name, Party, %Votes they had, Dollars raised and Dollars Spent and figuring out if they are Winner or incumbent and putting 'Yes' if the expression is a MAtch  or 'No' if there is no match. We are creating empty list and appending the list when it moves from member to member.

In [5]:
Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []


for list_item1 in soup.findAll('div', attrs={'class' : 'Members--list-item'}):
    
    list_item1 = list_item1.text

    list_item1 = list_item1.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
    
    
    Name.append(re.search(r"(.+)\(([A-Z])\)", list_item1)[1])
   
    Party.append(re.search(r"(.+)\(([A-Z])\)", list_item1)[2])
    
    pct_votes.append(re.search("\d+\.\d+\%", list_item1)[0])
    
    Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_item1)[1])
    
    Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_item1)[1])
    
    x = re.search(r"\bWinner", list_item1)  
    #the search makes the word match so it stops if no match so that is why weused the append YEs or No
    if x:
        Winner.append("Yes")
    else:
        Winner.append("No")
        
    y = re.search(r"\bIncumbent", list_item1)

    if y:
      Incumbent.append("Yes")
    else:
       Incumbent.append("No") 
print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent)

['   Mark Green ', '             Kiran Sreepada ', '             Ronald Brown ', '             Scott Vieira Jr '] ['R', 'D', 'I', 'I'] ['69.9%', '27.3%', '2.2%', '0.6%'] ['1194960', '206644', '1750', '655'] ['935487', '207191', '0', '1049'] ['Yes', 'No', 'No', 'No'] ['Yes', 'No', 'No', 'No']


#### Now converting these list into a Dictionary object and using that to creat the Data Frame

In [6]:
df_TN7 = {'CandidateName': Name,'Party': Party,  'PercentVotes' : pct_votes, 'FundsRaised': Raised, 'MoneySpent':Spent, 
          'Winner': Winner, 'Incumbent': Incumbent}
pd.DataFrame(df_TN7)

,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,Mark Green,R,69.9%,1194960,935487,Yes,Yes
1,Kiran Sreepada,D,27.3%,206644,207191,No,No
2,Ronald Brown,I,2.2%,1750,0,No,No
3,Scott Vieira Jr,I,0.6%,655,1049,No,No


### Trying to tanslate the above code for other districts in TN

#### There is pattern in website were 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N' where the id=TN is the stae and 07 is the district so we are going to follow the logic and creat a variabe called state feed the state code to it and a d_num variable which will change adding 1 from the previos district number and starts with 01

In [7]:
state = 'TN'
d_num = 1

#this URL is different for races/summary we need the races/candidates instead for picking the candidates
#URLX = 'https://www.opensecrets.org/races/summary?cycle=2022&id='+str(state)+'0'+str(d_num)+'&spec=N'  #URL of the webpage
#URLX = 'https://www.opensecrets.org/races/summary?cycle=2022&id=TN02&spec=N'

#URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N'
URLOX1 = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N'

URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+str(state)+'0'+str(d_num)+'&spec=N'
print(URLOX)

responseX = requests.get(URLOX)  #creating a response object pulling the URL
type(responseX)       # it is a requests.status.models.Response

responseX.status_code  #200 not broke read properly
responseX.text         # helps open the object to see it looks
soupX = BS(responseX.text, 'html.parser') #converts the response object in Beautiful object
#print(soup.prettify())
soupX.find('title')

list_itemX = soupX.find('div', attrs={'class' : 'Members--list-item'}).text
#list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")

list_itemX



https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N


'\n\n\n\t\t\t\t\t\t\t\t\t\tDiana Harshbarger (R)\n • Winner \n(74.8% of vote)\n\n\n\n\nRaised:\n$2,126,946\n\n\n\n\n\n\n\nSpent:\n$1,869,100\n\n\n\n\n\n\n\nCash on Hand:\n$257,846\n\n\n\n\n\n\n\n\nSource of Funds\n\n\n\nType of Contribution\nAmount\nPercentage\n\n\n\n\nSmall Individual Contributions (≤ $200)\n$44,239\n2.08%\n\n\nLarge Individual Contributions\n$315,489\n14.83%\n\n\nPAC Contributions*\n$222,800\n10.48%\n\n\nCandidate self-financing\n$1,461,293\n68.70%\n\n\nOther\n$83,124\n3.91%\n\n\n\nQuality of Disclosure\n\n\n\nType of Disclosure\nAmount\nPercentage\n\n\n\n\nFull Disclosure\n$185,473\n94.33%\n\n\nIncomplete Disclosure\n$2,800\n1.42%\n\n\nNo Disclosure\n$8,350\n4.25%\n\n\n\n\n'

In [8]:
Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []


for list_itemX in soupX.findAll('div', attrs={'class' : 'Members--list-item'}):
    
    list_itemX = list_itemX.text

    list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
    
    
    Name.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[1])
   
    Party.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[2])
    
    pct_votes.append(re.search("\d+\.\d+\%", list_itemX)[0])
    
    Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_itemX)[1])
    
    Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_itemX)[1])
    
    x = re.search(r"\bWinner", list_itemX)  
    #the search makes the word match so it stops if no match so that is why weused the append YEs or No
    if x:
        Winner.append("Yes")
    else:
        Winner.append("No")
        
    y = re.search(r"\bIncumbent", list_itemX)

    if y:
      Incumbent.append("Yes")
    else:
       Incumbent.append("No") 
print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent)

df_TN1 = {'CandidateName': Name,'Party': Party,  'PercentVotes' : pct_votes, 'FundsRaised': Raised, 'MoneySpent':Spent, 
          'Winner': Winner, 'Incumbent': Incumbent}
TN_01 = pd.DataFrame(df_TN1)
TN_01.head()



['             Diana Harshbarger ', '             Blair Nicole Walsingham '] ['R', 'D'] ['74.8%', '22.4%'] ['2126946', '140209'] ['1869100', '134995'] ['Yes', 'No'] ['No', 'No']


,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,Diana Harshbarger,R,74.8%,2126946,1869100,Yes,No
1,Blair Nicole Walsingham,D,22.4%,140209,134995,No,No


## Automating the system for reading all the districts and creating the Soup

In [9]:

d = soupX.find('ol',attrs = {'class':'SubNav-items'}) 
a_tags = d.find_all('a')
a_tags

d_count = 0                               #Start counting districts in state
for i in a_tags:
    if str.isdigit(i.text.strip()[-1]):   #if the last substring is a digit
        d_count+=1                        #increase district count by 1
    #print(i.text.strip()[-1])
    
d_count

9

In [10]:
state = 'TN'

count = 1

Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []



while count <= d_count:     #for each district number
    url = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+str(state)+'0'+str(count)+ '&spec=N'   #here's the url
    r = requests.get(url)    
    #soup = BS(r.text,'html.parser')
    soupX = BS(r.text)
    
    print(url)
    
    Name = []
    Party = []
    pct_votes = []
    Raised = []
    Spent = []
    Winner = []
    Incumbent = []
    
    for list_itemY in soupX.find('div', attrs={'class' : 'Members--list-item'}):
    
        list_itemY = list_itemY.text

        list_itemY = list_itemY.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")    

        type(list_itemY)
        
    count+=1 # increment count




https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN02&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN03&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN04&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN05&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN08&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN09&spec=N


In [11]:
xyz = soupX.find('div', attrs={'class' : 'Members--list-item'})
xyz = (xyz.text).replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
Name = re.search(r"(.+)\(([A-Z])\)", xyz)[1]
type(xyz)
xyz
Name

'   Steve Cohen '

In [12]:
URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N'

URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+str(state)+str(d_num)+'&spec=N'
print(URLOX)

https://www.opensecrets.org/races/candidates?cycle=2020&id=TN1&spec=N


In [13]:
state = 'TN'
d_num = 2

#this URL is different for races/summary we need the races/candidates instead for picking the candidates
#URLX = 'https://www.opensecrets.org/races/summary?cycle=2022&id='+str(state)+'0'+str(d_num)+'&spec=N'  #URL of the webpage
#URLX = 'https://www.opensecrets.org/races/summary?cycle=2022&id=TN02&spec=N'

#URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N'
URLOX1 = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N'

URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+str(state)+'0'+str(d_num)+'&spec=N'
print(URLOX)

responseX = requests.get(URLOX)  #creating a response object pulling the URL
type(responseX)       # it is a requests.status.models.Response

responseX.status_code  #200 not broke read properly
responseX.text         # helps open the object to see it looks
soupX = BS(responseX.text, 'html.parser') #converts the response object in Beautiful object
#print(soup.prettify())
soupX.find('title')

list_itemX = soupX.find('div', attrs={'class' : 'Members--list-item'}).text
#list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")

list_itemX


https://www.opensecrets.org/races/candidates?cycle=2020&id=TN02&spec=N


'\n\n\nTim Burchett (R) • Incumbent\n • Winner \n(67.7% of vote)\n\n\n\n\nRaised:\n$1,336,276\n\n\n\n\n\n\n\nSpent:\n$878,488\n\n\n\n\n\n\n\nCash on Hand:\n$593,678\n\n\n\n\n\n\n\n\nSource of Funds\n\n\n\nType of Contribution\nAmount\nPercentage\n\n\n\n\nSmall Individual Contributions (≤ $200)\n$343,014\n25.67%\n\n\nLarge Individual Contributions\n$729,831\n54.62%\n\n\nPAC Contributions*\n$269,535\n20.17%\n\n\nCandidate self-financing\n$0\n0.00%\n\n\nOther\n-$6,105\n-0.46%\n\n\n\nQuality of Disclosure\n\n\n\nType of Disclosure\nAmount\nPercentage\n\n\n\n\nFull Disclosure\n$708,346\n99.09%\n\n\nIncomplete Disclosure\n$5,250\n0.73%\n\n\nNo Disclosure\n$1,250\n0.17%\n\n\n\n\n'

In [14]:
Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []


for list_itemX in soupX.findAll('div', attrs={'class' : 'Members--list-item'}):
    
    list_itemX = list_itemX.text

    list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
    
    
    Name.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[1])
   
    Party.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[2])
    
    pct_votes.append(re.search("\d+\.\d+\%", list_itemX)[0])
    
    Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_itemX)[1])
    
    Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_itemX)[1])
    
    x = re.search(r"\bWinner", list_itemX)  
    #the search makes the word match so it stops if no match so that is why weused the append YEs or No
    if x:
        Winner.append("Yes")
    else:
        Winner.append("No")
        
    y = re.search(r"\bIncumbent", list_itemX)

    if y:
      Incumbent.append("Yes")
    else:
       Incumbent.append("No") 
print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent)

df_TN2 = {'CandidateName': Name,'Party': Party,  'PercentVotes' : pct_votes, 'FundsRaised': Raised, 'MoneySpent':Spent, 
          'Winner': Winner, 'Incumbent': Incumbent}
TN_02 = pd.DataFrame(df_TN1)
TN_02.head()


['   Tim Burchett ', '             Renee Hoyos '] ['R', 'D'] ['67.7%', '31.0%'] ['1336276', '812784'] ['878488', '816793'] ['Yes', 'No'] ['Yes', 'No']


,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,Diana Harshbarger,R,74.8%,2126946,1869100,Yes,No
1,Blair Nicole Walsingham,D,22.4%,140209,134995,No,No


In [15]:
state = {'TN': 9, 'CAL':53}

for key, value in state.items():
    for i in range(1, value+1):
        print(str(key) + " " + str(i))

TN 1
TN 2
TN 3
TN 4
TN 5
TN 6
TN 7
TN 8
TN 9
CAL 1
CAL 2
CAL 3
CAL 4
CAL 5
CAL 6
CAL 7
CAL 8
CAL 9
CAL 10
CAL 11
CAL 12
CAL 13
CAL 14
CAL 15
CAL 16
CAL 17
CAL 18
CAL 19
CAL 20
CAL 21
CAL 22
CAL 23
CAL 24
CAL 25
CAL 26
CAL 27
CAL 28
CAL 29
CAL 30
CAL 31
CAL 32
CAL 33
CAL 34
CAL 35
CAL 36
CAL 37
CAL 38
CAL 39
CAL 40
CAL 41
CAL 42
CAL 43
CAL 44
CAL 45
CAL 46
CAL 47
CAL 48
CAL 49
CAL 50
CAL 51
CAL 52
CAL 53


In [16]:
Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []
ST = []
ST_DIT = []


for key, value in state.items():
    for i in range(1, value+1):
        #print(str(key) + " " + str(i))
        url = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+ key+ f"{i:02}"+'&spec=N'
        #print(url)
        
        res1 = requests.get(url)  #creating a response object pulling the URL
        type(res1)       # it is a requests.status.models.Response

        res1.status_code  #200 not broke read properly
        res1.text         # helps open the object to see it looks
        soupX = BS(res1.text, 'html.parser') #converts the response object in Beautiful object
        #print(soup.prettify())
        soupX.find('title')

        #list_itemX = soupX.find('div', attrs={'class' : 'Members--list-item'}).text
        #list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")

        #print(list_itemX)
        for list_itemX in soupX.findAll('div', attrs={'class' : 'Members--list-item'}):
    
            list_itemX = list_itemX.text

            list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
            
            ST.append(key)
            ST_DIT.append(f"{i:02}")
    
    
            Name.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[1])
   
            Party.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[2])

            pct_votes.append(re.search("\d+\.\d+\%", list_itemX)[0])

            Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_itemX)[1])

            Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_itemX)[1])

            x = re.search(r"\bWinner", list_itemX)  
            #the search makes the word match so it stops if no match so that is why weused the append YEs or No
            if x:
                Winner.append("Yes")
            else:
                Winner.append("No")

            y = re.search(r"\bIncumbent", list_itemX)

            if y:
              Incumbent.append("Yes")
            else:
               Incumbent.append("No") 
#print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent, ST, ST_DIT)

dic_list = {'State':ST, 'District':ST_DIT, 'CandidateName': Name,'Party': Party,  'PercentVotes' : pct_votes, 'FundsRaised': Raised, 'MoneySpent':Spent, 
          'Winner': Winner, 'Incumbent': Incumbent}
TN_CA = pd.DataFrame(dic_list)
TN_CA.head()

,State,District,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,TN,01,Diana Harshbarger,R,74.8%,2126946,1869100,Yes,No
1,TN,01,Blair Nicole Walsingham,D,22.4%,140209,134995,No,No
2,TN,02,Tim Burchett,R,67.7%,1336276,878488,Yes,Yes
3,TN,02,Renee Hoyos,D,31.0%,812784,816793,No,No
4,TN,03,Chuck Fleischmann,R,67.3%,1051653,381411,Yes,Yes


In [24]:
TN_CA.shape
TN_CA.tail(20)

,State,District,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
105,CAL,44,Ron Estes,R,64.0%,1993808,1420188,Yes,Yes
106,CAL,44,Laura Lombard,D,36.0%,264459,258684,No,No
107,CAL,45,Ron Estes,R,64.0%,1993808,1420188,Yes,Yes
108,CAL,45,Laura Lombard,D,36.0%,264459,258684,No,No
109,CAL,46,Ron Estes,R,64.0%,1993808,1420188,Yes,Yes
110,CAL,46,Laura Lombard,D,36.0%,264459,258684,No,No
111,CAL,47,Ron Estes,R,64.0%,1993808,1420188,Yes,Yes
112,CAL,47,Laura Lombard,D,36.0%,264459,258684,No,No
113,CAL,48,Ron Estes,R,64.0%,1993808,1420188,Yes,Yes
114,CAL,48,Laura Lombard,D,36.0%,264459,258684,No,No


In [25]:
TN_CA.to_csv('TN_CAL.csv')

In [18]:
url_state = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'
r_state = requests.get(url_state)
soup_state = BS(r_state.text,'html.parser')

co_table = pd.read_html(str(soup_state.find('table')))
co_table_df = co_table[0]
co_table_df = co_table_df.drop(co_table_df.tail(1).index)
co_table_df.head()

,state,representatives
0,Alabama,7
1,Alaska,1
2,Arizona,9
3,Arkansas,4
4,California,53


In [19]:
state_abb = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ",
    "Arkansas": "AR", "California": "CA","Colorado": "CO",
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL",
    "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA",
    "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA",
    "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS",
    "Missouri": "MO","Montana": "MT", "Nebraska": "NE",
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC",
    "North Dakota": "ND", "Ohio": "OH","Oklahoma": "OK",
    "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI",
    "South Carolina": "SC", "South Dakota": "SD",
    "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY"
    }

In [20]:
co_table_df['state_code'] = co_table_df.state.map(state_abb)
co_table_df.head()

,state,representatives,state_code
0,Alabama,7,AL
1,Alaska,1,AK
2,Arizona,9,AZ
3,Arkansas,4,AR
4,California,53,CA


In [21]:
state_rep = dict(zip(co_table_df.state_code, co_table_df.representatives))
state_rep

{'AL': 7,
 'AK': 1,
 'AZ': 9,
 'AR': 4,
 'CA': 53,
 'CO': 7,
 'CT': 5,
 'DE': 1,
 'FL': 27,
 'GA': 14,
 'HI': 2,
 'ID': 2,
 'IL': 18,
 'IN': 9,
 'IA': 4,
 'KS': 4,
 'KY': 6,
 'LA': 6,
 'ME': 2,
 'MD': 8,
 'MA': 9,
 'MI': 14,
 'MN': 8,
 'MS': 4,
 'MO': 8,
 'MT': 1,
 'NE': 3,
 'NV': 4,
 'NH': 2,
 'NJ': 12,
 'NM': 3,
 'NY': 27,
 'NC': 13,
 'ND': 1,
 'OH': 16,
 'OK': 5,
 'OR': 5,
 'PA': 18,
 'RI': 2,
 'SC': 7,
 'SD': 1,
 'TN': 9,
 'TX': 36,
 'UT': 4,
 'VT': 1,
 'VA': 11,
 'WA': 10,
 'WV': 3,
 'WI': 8,
 'WY': 1}

You can achieve this using formatted string
num = 1
number = f"{num:02}"
--------------
output:
"01"
#'CA': 53,


In [22]:
state_big = {'AL': 7, 'AK': 1, 'AZ': 9, 'AR': 4,   'DE': 1, 'FL': 27,  'ID': 2, 'IN': 9, 'IA': 4, 'KS': 4, 'KY': 6, 'LA': 6, 'ME': 2, 'MD': 8, 'MA': 9,   'MS': 4, 'MO': 8, 'MT': 1, 'NE': 3, 'NV': 4, 'NH': 2, 'NJ': 12, 'NM': 3, 'NC': 13, 'ND': 1, 'OH': 16, 
             'OR': 5, 'PA': 18, 'RI': 2, 'SC': 7, 'SD': 1, 'TN': 9, 'TX': 36, 'VT': 1, 'VA': 11, 'WA': 10, 'WV': 3,
             'WI': 8, 'WY': 1}
#done{'AL': 7, 'AK': 1, 'AZ': 9, 'AR': 4,  'CT': 5, 'DE': 1, 'FL': 27,  'ID': 2, 'IN': 9, 'IA': 4,}
#error{'CA': 53, 'CO': 7, 'CT': 5, 'GA': 14, 'HI': 2, 'IL': 18, 'MI': 14, 'MN': 8, 'NY': 27, 'OK': 5, 'UT': 4, }

#CA is part of the TN_CA df as 

In [44]:

state_A = {'AL': 7, 'AK': 1, 'AZ': 9, 'AR': 4}
state_C = {'CA': 53, 'CO': 7, 'CT': 5}
state_DF = {'DE': 1, 'FL': 27}
state_GH = {'GA': 14, 'HI': 2}
state_I = {'ID': 2, 'IN': 9, 'IA': 4}

notdone = {'CO': 7, 'CT': 5, 'GA': 14, 'HI': 2, 'IL': 18, 'MI': 14, 'MN': 8, 'NY': 27, 'OK': 5, 'UT': 4}
Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []
ST = []
ST_DIT = []
failedurls = []


for key, value in state_big.items():
    for i in range(1, value+1):
        #print(str(key) + " " + str(i))
        url = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+ key+ f"{i:02}"+'&spec=N'
        print(url)
        
        res1 = requests.get(url)  #creating a response object pulling the URL
        #type(res1)       # it is a requests.status.models.Response
              
        retry_count = 0
        next_url = False
        while res1.status_code != 200:
            time.sleep(5)
            if retry_count > 10:
                failedurls.append(url)
                next_url = True
                break
            res1 = requests.get(url)
            retry_count +=1
        
        if next_url:
            continue
        #soup = BS(r.text,'html.parser')
    
        #print(url)      
        

        #res1.status_code  #200 not broken read properly
        #res1.text         # helps open the object to see it looks
        soupX = BS(res1.text, 'html.parser') #converts the response object in Beautiful object
        #print(soup.prettify())
        soupX.find('title')

        #list_itemX = soupX.find('div', attrs={'class' : 'Members--list-item'}).text
        #list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")

        #print(list_itemX)
        for list_itemX in soupX.findAll('div', attrs={'class' : 'Members--list-item'}):
    
            list_itemX = list_itemX.text

            list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
            
            ST.append(key)
            ST_DIT.append(f"{i:02}")
    
    
            Name.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[1])
   
            Party.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[2])

            pct_votes.append(re.search("\d+\.\d+\%", list_itemX)[0])

            Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_itemX)[1])

            Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_itemX)[1])

            x = re.search(r"\bWinner", list_itemX)  
            #the search makes the word match so it stops if no match so that is why weused the append YEs or No
            if x:
                Winner.append("Yes")
            else:
                Winner.append("No")

            y = re.search(r"\bIncumbent", list_itemX)

            if y:
              Incumbent.append("Yes")
            else:
               Incumbent.append("No") 
#print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent, ST, ST_DIT)

dic_list = {'State':ST, 'District':ST_DIT, 'CandidateName': Name,'Party': Party,  'PercentVotes' : pct_votes, 'FundsRaised': Raised, 'MoneySpent':Spent, 
          'Winner': Winner, 'Incumbent': Incumbent}
USA_done = pd.DataFrame(dic_list)

USA_done.head()

https://www.opensecrets.org/races/candidates?cycle=2020&id=AL01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AL02&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AL03&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AL04&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AL05&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AL06&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AL07&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AK01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ02&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ03&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ04&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ05&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ06&spec=N
https:

https://www.opensecrets.org/races/candidates?cycle=2020&id=NV03&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NV04&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NH01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NH02&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ02&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ03&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ04&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ05&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ06&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ07&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ08&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ09&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=NJ10&spec=N
https:

https://www.opensecrets.org/races/candidates?cycle=2020&id=TX28&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TX29&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TX30&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TX31&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TX32&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TX33&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TX34&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TX35&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TX36&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=VT01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=VA01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=VA02&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=VA03&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=VA04&spec=N
https:

,State,District,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,AL,01,Jerry Carl,R,64.9%,1971321,1859349,Yes,No
1,AL,01,James Averhart,D,35.0%,80095,78973,No,No
2,AL,02,Barry Moore,R,65.3%,650807,669368,Yes,No
3,AL,02,Phyllis Harvey-Hall,D,34.6%,56050,55988,No,No
4,AL,03,Mike D Rogers,R,67.5%,1193111,1218564,Yes,Yes


In [46]:
#USA_big.shape 

USA_done.shape

(566, 9)

In [48]:
USA_done

,State,District,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,AL,01,Jerry Carl,R,64.9%,1971321,1859349,Yes,No
1,AL,01,James Averhart,D,35.0%,80095,78973,No,No
2,AL,02,Barry Moore,R,65.3%,650807,669368,Yes,No
3,AL,02,Phyllis Harvey-Hall,D,34.6%,56050,55988,No,No
4,AL,03,Mike D Rogers,R,67.5%,1193111,1218564,Yes,Yes
...,...,...,...,...,...,...,...,...,...
561,WI,07,Tricia Zunker,D,39.2%,1261957,1232690,No,No
562,WI,08,Mike Gallagher,R,64.0%,3202905,2841801,Yes,Yes
563,WI,08,Amanda Stuck,D,36.0%,416978,399916,No,No
564,WY,01,Liz Cheney,R,68.6%,3003883,3060167,Yes,Yes


In [49]:
USA_done.to_csv('USA_done.csv')

## The big df is created for the done states, but some districts are giving trouble states are giving. Going to test them
**Done states = {'AL': 7, 'AK': 1, 'AZ': 9, 'AR': 4,   'DE': 1, 'FL': 27,  'ID': 2, 'IN': 9, 'IA': 4, 'KS': 4, 'KY': 6, 'LA': 6, 'ME': 2, 'MD': 8, 'MA': 9,   'MS': 4, 'MO': 8, 'MT': 1, 'NE': 3, 'NV': 4, 'NH': 2, 'NJ': 12, 'NM': 3, 'NC': 13, 'ND': 1, 'OH': 16, 'OR': 5, 'PA': 18, 'RI': 2, 'SC': 7, 'SD': 1, 'TN': 9, 'TX': 36, 'VT': 1, 'VA': 11, 'WA': 10, 'WV': 3,
             'WI': 8, 'WY': 1} saved as the USA_done.csv****
 **CA is part of TN_CAL.csv**

***not done #error{'CA': 53, 'CO': 7, 'CT': 5, 'GA': 14, 'HI': 2, 'IL': 18, 'MI': 14, 'MN': 8, 'NY': 27, 'OK': 5, 'UT': 4}***



In [ ]:
state = 'HI'
d_num = 2

#this URL is different for races/summary we need the races/candidates instead for picking the candidates
URLOX1 = 'https://www.opensecrets.org/races/summary?cycle=2022&id='+str(state)+'0'+str(d_num)+'&spec=N'  #URL of the webpage
#URLX = 'https://www.opensecrets.org/races/summary?cycle=2022&id=TN02&spec=N'


#URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+str(state)+'0'+str(d_num)+'&spec=N'
print(URLOX1)

responseX1 = requests.get(URLOX1)  #creating a response object pulling the URL
type(responseX1)       # it is a requests.status.models.Response

responseX1.status_code  #200 not broke read properly
responseX1.text         # helps open the object to see it looks
soupX1 = BS(responseX1.text, 'html.parser') #converts the response object in Beautiful object
#print(soup.prettify())
soupX1.find('title')

list_itemX1 = soupX1.find('div', attrs={'class' : 'Members--list-item'}).text
#list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")

list_itemX1
list_itemX1 = list_itemX1.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
print(list_item1)



***not done #error{'CA': 53, 'CO': 7, 'CT': 5, 'GA': 14, 'HI': 2, 'IL': 18, 'MI': 14, 'MN': 8, 'NY': 27, 'OK': 5, 'UT': 4***

***Problem districts: CT03,
                    CO03, CO05, CO06***

In [50]:
#not done #error{'CA': 53, 'CO': 7, 'CT': 5, 'GA': 14, 'HI': 2, 'IL': 18, 'MI': 14, 'MN': 8, 'NY': 27, 'OK': 5, 'UT': 4}

state = 'UT'
d_num = 4

#URLOX1 = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=CO01&spec=N'

URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+str(state)+'0'+str(d_num)+'&spec=N'
print(URLOX)

responseX1 = requests.get(URLOX)  #creating a response object pulling the URL
type(responseX1)       # it is a requests.status.models.Response

responseX1.status_code  #200 not broke read properly
responseX1.text         # helps open the object to see it looks
soupX1 = BS(responseX1.text, 'html.parser') #converts the response object in Beautiful object
#print(soup.prettify())
soupX1.find('title')



Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []
ST = []
ST_DIT = []


for list_itemX1 in soupX1.findAll('div', attrs={'class' : 'Members--list-item'}):
    
    list_itemX1 = list_itemX1.text

    list_itemX1 = list_itemX1.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
    
    
    Name.append(re.search(r"(.+)\(([A-Z])\)", list_itemX1)[1])
   
    Party.append(re.search(r"(.+)\(([A-Z])\)", list_itemX1)[2])
    
    pct_votes.append(re.search("\d+\.\d+\%", list_itemX1)[0])
    
    Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_itemX1)[1])
    
    Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_itemX1)[1])
    
    x = re.search(r"\bWinner", list_itemX1)  
    #the search makes the word match so it stops if no match so that is why weused the append YEs or No
    if x:
        Winner.append("Yes")
    else:
        Winner.append("No")
        
    y = re.search(r"\bIncumbent", list_itemX1)

    if y:
      Incumbent.append("Yes")
    else:
       Incumbent.append("No") 
print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent)

dic = {'CandidateName': Name,'Party': Party,  'PercentVotes' : pct_votes, 'FundsRaised': Raised, 'MoneySpent':Spent, 
          'Winner': Winner, 'Incumbent': Incumbent}
df = pd.DataFrame(dic)
df.head()

https://www.opensecrets.org/races/candidates?cycle=2020&id=UT04&spec=N


TypeError: 'NoneType' object is not subscriptable

In [81]:
state = 'UT'
d_num = 4

Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []

#URLOX1 = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=CO01&spec=N'

URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+str(state)+'0'+str(d_num)+'&spec=N'
print(URLOX)

responseX1 = requests.get(URLOX)  #creating a response object pulling the URL
type(responseX1)       # it is a requests.status.models.Response

responseX1.status_code  #200 not broke read properly
responseX1.text         # helps open the object to see it looks
soupX1 = BS(responseX1.text) #converts the response object in Beautiful object
#print(soup.prettify())
soupX1.find('title')
for list_itemX1 in soupX1.findAll('div', attrs={'class' : 'Members--list-item'}):
    
    list_itemX1 = list_itemX1.text
    print(list_itemX1)
    list_itemX2 = list_itemX1.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
    print(list_itemX2)
    Name.append(re.search(r"(.+)\(([A-Z])\)", list_itemX2)[1])
   
    Party.append(re.search(r"(.+)\(([A-Z])\)", list_itemX2)[2])
    
    pct_votes.append(re.search("\d+\.\d+\%", list_itemX2)[0])
    
    Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_itemX2)[1])
    
    Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_itemX2)[1])
    
print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent)

https://www.opensecrets.org/races/candidates?cycle=2020&id=UT04&spec=N



Ben McAdams (D) • Incumbent

(46.7% of vote)




Raised:
$5,613,542







Spent:
$5,555,591







Cash on Hand:
$136,323








Source of Funds



Type of Contribution
Amount
Percentage




Small Individual Contributions (≤ $200)
$632,670
11.27%


Large Individual Contributions
$3,477,897
61.96%


PAC Contributions*
$1,518,054
27.04%


Candidate self-financing
$0
0.00%


Other
-$15,079
-0.27%



Quality of Disclosure



Type of Disclosure
Amount
Percentage




Full Disclosure
$2,033,665
94.95%


Incomplete Disclosure
$35,941
1.68%


No Disclosure
$72,326
3.38%





   Ben McAdams (D)  Incumbent  (46.7% of vote)     Raised: $5613542        Spent: $5555591        Cash on Hand: $136323         Source of Funds    Type of Contribution Amount Percentage     Small Individual Contributions (≤ $200) $632670 11.27%   Large Individual Contributions $3477897 61.96%   PAC Contributions* $1518054 27.04%   Candidate self-fin

TypeError: 'NoneType' object is not subscriptable

In [96]:
#notdone = {'CA':53, 'CO': 7, 'CT': 5, 'GA': 14, 'HI': 2, 'IL': 18, 'MI': 14, 'MN': 8, 'NY': 27, 'OK': 5, 'UT': 4}
notdone = {'CA': 28}

Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []
ST = []
ST_DIT = []
failedurls = []


for key, value in notdone.items():
    for i in range(1, value+1):
        #print(str(key) + " " + str(i))
        url = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+ key+ f"{i:02}"+'&spec=N'
        print(url)
        
        res1 = requests.get(url)  #creating a response object pulling the URL
        #type(res1)       # it is a requests.status.models.Response
              
        retry_count = 0
        next_url = False
        while res1.status_code != 200:
            time.sleep(5)
            if retry_count > 10:
                failedurls.append(url)
                next_url = True
                break
            res1 = requests.get(url)
            retry_count +=1
        
        if next_url:
            continue
        #soup = BS(r.text,'html.parser')
    
        #print(url)      
        

        #res1.status_code  #200 not broken read properly
        #res1.text         # helps open the object to see it looks
        soupX = BS(res1.text, 'html.parser') #converts the response object in Beautiful object
        #print(soup.prettify())
        soupX.find('title')

        #list_itemX = soupX.find('div', attrs={'class' : 'Members--list-item'}).text
        #list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")

        #print(list_itemX)
        for list_itemX in soupX.findAll('div', attrs={'class' : 'Members--list-item'}):
    
            list_itemX = list_itemX.text

            list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
            
            ST.append(key)
            ST_DIT.append(f"{i:02}")
    
    
            Name.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[1])
   
            Party.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[2])

            pct_votes.append(re.search("\d+\.\d+\%", list_itemX)[0])

            Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_itemX)[1])

            Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_itemX)[1])

            x = re.search(r"\bWinner", list_itemX)  
            #the search makes the word match so it stops if no match so that is why weused the append YEs or No
            if x:
                Winner.append("Yes")
            else:
                Winner.append("No")

            y = re.search(r"\bIncumbent", list_itemX)

            if y:
              Incumbent.append("Yes")
            else:
               Incumbent.append("No") 
#print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent, ST, ST_DIT)

dic_list = {'State':ST, 'District':ST_DIT, 'CandidateName': Name,'Party': Party,  'PercentVotes' : pct_votes, 'FundsRaised': Raised, 'MoneySpent':Spent, 
          'Winner': Winner, 'Incumbent': Incumbent}
not_done = pd.DataFrame(dic_list)
not_done.head()

https://www.opensecrets.org/races/candidates?cycle=2020&id=CA01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA02&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA03&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA04&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA05&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA06&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA07&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA08&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA09&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA10&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA11&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA12&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA13&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=CA14&spec=N
https:

,State,District,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,CA,01,Doug LaMalfa,R,56.9%,1219556,1155945,Yes,Yes
1,CA,01,Audrey Denney,D,43.1%,2461769,2457661,No,No
2,CA,02,Jared Huffman,D,75.7%,899419,797753,Yes,Yes
3,CA,02,Dale Mensing,R,24.3%,5921,2197,No,No
4,CA,03,John Garamendi,D,54.7%,1054066,667320,Yes,Yes


In [97]:
not_done.shape

(53, 9)

In [98]:
not_done.head(20)

,State,District,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,CA,01,Doug LaMalfa,R,56.9%,1219556,1155945,Yes,Yes
1,CA,01,Audrey Denney,D,43.1%,2461769,2457661,No,No
2,CA,02,Jared Huffman,D,75.7%,899419,797753,Yes,Yes
3,CA,02,Dale Mensing,R,24.3%,5921,2197,No,No
4,CA,03,John Garamendi,D,54.7%,1054066,667320,Yes,Yes
5,CA,03,Tamika Hamilton,R,45.3%,397099,384584,No,No
6,CA,04,Tom McClintock,R,55.9%,2053363,2107248,Yes,Yes
7,CA,04,Brynne Kennedy,D,44.1%,3006020,3002532,No,No
8,CA,05,Mike Thompson,D,76.8%,2208136,2022751,Yes,Yes
9,CA,06,Doris Matsui,D,73.4%,1019605,1073529,Yes,Yes


In [100]:
not_done.to_csv('CA_28.csv')